In [1]:
import numpy as np
from scipy import io
import tensorflow as tf
import os
import time
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import math

In [2]:
snr_0  = h5py.File('../Data/VAD_Training_data_SNR_0.mat')
snr_5  = h5py.File('../Data/VAD_Training_data_SNR_5.mat')
snr_10 = h5py.File('../Data/VAD_Training_data_SNR_10.mat')

In [3]:
data   = np.empty([0,40,40])
labels = np.empty([0,2])

data = np.r_[data, np.transpose(snr_0['trainingData'].value, axes=(2,0,1))]
data = np.r_[data, np.transpose(snr_5['trainingData'].value, axes=(2,0,1))]
data = np.r_[data, np.transpose(snr_10['trainingData'].value, axes=(2,0,1))]
data = np.r_[data, np.transpose(snr_0['testingData'].value, axes=(2,0,1))]
data = np.r_[data, np.transpose(snr_5['testingData'].value, axes=(2,0,1))]
data = np.r_[data, np.transpose(snr_10['testingData'].value, axes=(2,0,1))]

labels = np.r_[labels, np.transpose(snr_0['trainingLabels'].value)]
labels = np.r_[labels, np.transpose(snr_5['trainingLabels'].value)]
labels = np.r_[labels, np.transpose(snr_10['trainingLabels'].value)]
labels = np.r_[labels, np.transpose(snr_0['testingLabels'].value)]
labels = np.r_[labels, np.transpose(snr_5['testingLabels'].value)]
labels = np.r_[labels, np.transpose(snr_10['testingLabels'].value)]

print("Data Shape ", data.shape)
print("Labels Shape", labels.shape)

Data Shape  (1094400, 40, 40)
Labels Shape (1094400, 2)


In [4]:
K = 40
L = 20
M = 10
N = 100
nClasses = 2
div = 10

tf.reset_default_graph()

with tf.name_scope("hyperparameters"):
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)

with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32,[None, 40, 40], "x-input")    
    x_image = tf.reshape(x, [-1, 40, 40, 1], name="x-image")    
    Y_ = tf.placeholder(tf.float32, [None, nClasses], "y-input")    
    
with tf.name_scope("model"):
    W1 = tf.Variable(tf.truncated_normal([5,5,1,K], stddev=0.05))
    B1 = tf.Variable(tf.ones([K])/div)
    Y1 = tf.nn.relu(tf.nn.conv2d(x_image, W1, strides=[1,2,2,1], padding='SAME') + B1)
    
    W2 = tf.Variable(tf.truncated_normal([5,5,K,L], stddev=0.05))
    B2 = tf.Variable(tf.ones([L])/div)
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1,2,2,1], padding='SAME') + B2)
    
    W3 = tf.Variable(tf.truncated_normal([5,5,L,M], stddev=0.05))
    B3 = tf.Variable(tf.ones([M])/div)
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1,2,2,1], padding='SAME') + B3)
    
    YY = tf.reshape(Y3, shape=[-1, 5*5*M])   
    
    W4 = tf.Variable(tf.truncated_normal([5*5*M, N], stddev=0.05))
    B4 = tf.Variable(tf.ones([N])/div)
    Yf = tf.nn.relu(tf.matmul(YY, W4) + B4)
    Y4 = tf.nn.dropout(Yf, keep_prob)
    
    W5 = tf.Variable(tf.truncated_normal([N, 2], stddev=0.05))
    B5 = tf.Variable(tf.ones([2])/div)
    Y  = tf.nn.softmax(tf.matmul(Y4, W5) + B5)
    
with tf.name_scope("loss"):
    cross_entropy = -tf.reduce_sum(Y_*tf.log(tf.clip_by_value(Y,1e-10,1.0)))
    is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op  = optimizer.minimize(cross_entropy) 

In [5]:
checkpoint_dir = "full_model/"
tf.gfile.MakeDirs(checkpoint_dir)

learningRates = np.hstack((1e-3*np.ones(6),
                           1e-4*np.ones(4),
                           1e-5*np.ones(2)))
nEpochs = len(learningRates)
trainBatchSize = 512
nDataSamples = len(data)

with tf.Session() as sess:
    saver = tf.train.Saver()
    tf.train.write_graph(sess.graph_def,
                         checkpoint_dir,
                         "graph.pbtxt",
                         True)
    sess.run(tf.global_variables_initializer())    
    
    for epoch in np.arange(nEpochs):
        idx = np.arange(len(labels))
        np.random.shuffle(idx)
        
        for i in np.arange(0, nDataSamples, trainBatchSize):
            x_batch = data[idx[i:i+trainBatchSize], :, :]
            y_batch = labels[idx[i:i+trainBatchSize]]
            
            feed = {x: x_batch,
                    Y_: y_batch,
                   learning_rate: learningRates[epoch],
                   keep_prob: 0.75}
            sess.run(train_op, feed_dict=feed)
            
            if i%50*trainBatchSize == 0:
                feed = {x: x_batch,
                        Y_: y_batch,
                        learning_rate: learningRates[epoch],
                        keep_prob: 1.0}
                
                train_accuracy, loss_value = sess.run([accuracy, cross_entropy], feed_dict=feed)
                print("epoch: %2d step: %6d, Training Accuracy: %3.2f, loss: %6.4f" % \
                      (epoch, i, train_accuracy*100, loss_value))
                
        tf.gfile.MakeDirs(checkpoint_dir + '/model' + str(epoch))
        checkpoint_file = os.path.join(checkpoint_dir + '/model' + str(epoch), "model")
        saver.save(sess, checkpoint_file)
        print("**** SAVED MODEL ****")      
        print("**** COMPLETED EPOCH ****")

epoch:  0 step:      0, Training Accuracy: 75.20, loss: 306.3640
epoch:  0 step:  12800, Training Accuracy: 87.50, loss: 184.5056
epoch:  0 step:  25600, Training Accuracy: 93.36, loss: 119.0420
epoch:  0 step:  38400, Training Accuracy: 92.97, loss: 111.5587
epoch:  0 step:  51200, Training Accuracy: 92.58, loss: 116.3568
epoch:  0 step:  64000, Training Accuracy: 93.95, loss: 87.4597
epoch:  0 step:  76800, Training Accuracy: 94.73, loss: 81.7672
epoch:  0 step:  89600, Training Accuracy: 95.90, loss: 78.5267
epoch:  0 step: 102400, Training Accuracy: 94.73, loss: 73.7936
epoch:  0 step: 115200, Training Accuracy: 94.92, loss: 78.0443
epoch:  0 step: 128000, Training Accuracy: 94.14, loss: 84.5261
epoch:  0 step: 140800, Training Accuracy: 94.34, loss: 92.8842
epoch:  0 step: 153600, Training Accuracy: 93.36, loss: 94.3452
epoch:  0 step: 166400, Training Accuracy: 96.88, loss: 58.3215
epoch:  0 step: 179200, Training Accuracy: 94.53, loss: 90.6597
epoch:  0 step: 192000, Training Ac